In [3]:
## inspect payload object
import json
from pprint import pprint




NameError: name 'payload' is not defined

In [4]:
# If you make edits to get_votes.py and want them picked up automatically:
%load_ext autoreload
%autoreload 2

# Standard imports
from get_votes import fetch_and_store_batch, build_vote_matrix, compute_member_total_scores
import pandas as pd




# 1) Define the bills you want to fetch:
bills = [
    (118, "hr", 8034),
    (118, "hr", 6090),
    (118, "hr", 340),
    (118, "hr", 6126),
    (118, "hr", 2670),
    (118, "hr", 7217),
    (118, "hr", 5917),
    (118, "hr", 8038),
    (118, "hr", 8369),
    (118, "hr", 8070),
    (118, "hr", 8771),
    (118, "hr", 8774),
    (118, "hr", 5961),
]





# 2) Fetch & store into votes.db (will create/append the SQLite file)
fetch_and_store_batch(bills, db_path="votes.db")

# 3) Build the vote‐matrix DataFrame
vote_matrix = build_vote_matrix(bills, db_path="votes.db")

# 4) Display it in the notebook





Payload keys: ['actions', 'pagination', 'request']
  Fetched 15 actions at offset 0
{'actionCode': 'H37100', 'actionDate': '2024-04-20', 'actionTime': '14:02:43', 'recordedVotes': [{'chamber': 'House', 'congress': 118, 'date': '2024-04-20T18:02:43Z', 'rollNumber': 152, 'sessionNumber': 2, 'url': 'https://clerk.house.gov/evs/2024/roll152.xml'}], 'sourceSystem': {'code': 2, 'name': 'House floor actions'}, 'text': 'On passage Passed by the Yeas and Nays: 366 - 58 (Roll no. 152). (text: CR H2607-2610)', 'type': 'Floor'}
Payload keys: ['actions', 'pagination', 'request']
  Fetched 17 actions at offset 0
{'actionCode': 'H37100', 'actionDate': '2024-05-01', 'actionTime': '16:55:30', 'recordedVotes': [{'chamber': 'House', 'congress': 118, 'date': '2024-05-01T20:55:30Z', 'rollNumber': 172, 'sessionNumber': 2, 'url': 'https://clerk.house.gov/evs/2024/roll172.xml'}], 'sourceSystem': {'code': 2, 'name': 'House floor actions'}, 'text': 'On passage Passed by the Yeas and Nays: 320 - 91 (Roll no. 172

In [5]:
# your raw weights list: (congress, bill_type, bill_number, yea, present, nay, no_vote)
weights = [
    (118, "hr", 8034, -1, 0,  1, 0),
    (118, "hr", 6090, -1, 0,  1, 0),
    (118, "hr", 340,  -1, 0,  1, 0),
    (118, "hr", 6126, -1, 0,  1, 0),
    (118, "hr", 2670, -1, 0,  1, 0),
    (118, "hr", 7217, -1, 0,  1, 0),
    (118, "hr", 2882, -1, 0,  1, 0),
    (118, "hr", 5917, -1, 0,  1, 0),
    (118, "hr", 8038, -1, 0,  1, 0),
    (118, "hr", 8369, -1, 0,  1, 0),
    (118, "hr", 8070, -1, 0,  1, 0),
    (118, "hr", 8771, -1, 0,  1, 0),
    (118, "hr", 8774, -1, 0,  1, 0),
    (118, "hr", 5961, -1, 0,  1, 0),
]

# Build the rules dict
rules = {}
for _, bill_type, bill_number, yea_w, pres_w, nay_w, no_w in weights:
    bill_id = f"{bill_type.upper()}.{bill_number}"
    rules[bill_id] = {
        "Yea":        yea_w,
        "Nay":        nay_w,
        "Present":    pres_w,
        "Not Voting": no_w,
    }

# Example output:
# {
#   "HR.8034": {"Yea": -1, "Nay": 1, "Present": 0, "Not Voting": 0},
#    … 
# }

# Then pass `rules` into your scorer:
scores = compute_member_total_scores(vote_matrix, rules, default_score=0.0)
print(scores.head())


                   name state party  total_score  rank  percent HR.2670  \
member_id                                                                 
T000481           Tlaib    MI     D         12.0     1      0.2     Nay   
O000173            Omar    MN     D         12.0     1      0.2     Nay   
V000081       Velázquez    NY     D         10.0    10      2.3     Nay   
R000617         Ramirez    IL     D         10.0    10      2.3     Nay   
J000309    Jackson (IL)    IL     D         10.0    10      2.3     Nay   

          HR.340 HR.5917 HR.5961 HR.6090 HR.6126 HR.7217 HR.8034 HR.8038  \
member_id                                                                  
T000481      Nay     Nay     Nay     Nay     Nay     Nay     Nay     Nay   
O000173      Nay     Nay     Nay     Nay     Nay     Nay     Nay     Nay   
V000081      Nay     Yea     Nay     Nay     Nay     Nay     Nay     Nay   
R000617      Nay     Yea     Nay     Nay     Nay     Nay     Nay     Nay   
J000309      Nay  

In [10]:
print(scores[scores["state"] == "NY"])


                    name state party  total_score  rank  percent     HR.2670  \
member_id                                                                      
V000081        Velázquez    NY     D         10.0    10      2.3         Nay   
O000172    Ocasio-Cortez    NY     D          9.0    18      4.1         Nay   
B001223           Bowman    NY     D          8.0    24      5.4         Nay   
C001067      Clarke (NY)    NY     D          6.0    41      9.3         Nay   
K000402          Kennedy    NY     D          3.0    77     17.5         NaN   
T000469            Tonko    NY     D          2.0   100     22.7         Nay   
M001137            Meeks    NY     D          2.0   100     22.7         Nay   
N000002           Nadler    NY     D          2.0   100     22.7         Nay   
H001038     Higgins (NY)    NY     D          2.0   100     22.7         Nay   
J000294         Jeffries    NY     D          0.0   152     34.5         Nay   
M001206          Morelle    NY     D    

,name,state,party,total_score,rank,percent,HR.2670,HR.340,HR.5917,HR.5961,HR.6090,HR.6126,HR.7217,HR.8034,HR.8038,HR.8070,HR.8369,HR.8771,HR.8774
member_id,,,,,,,,,,,,,,,,,,,
V000081,Velázquez,NY,D,10.0,10,2.3,Nay,Nay,Yea,Nay,Nay,Nay,Nay,Nay,Nay,No,Nay,Nay,Nay
O000172,Ocasio-Cortez,NY,D,9.0,18,4.1,Nay,Nay,Yea,Not Voting,Nay,Nay,Nay,Nay,Nay,No,Nay,Nay,Nay
B001223,Bowman,NY,D,8.0,24,5.4,Nay,Nay,Yea,Nay,Nay,Nay,Nay,Nay,Nay,Not Voting,Nay,Not Voting,Not Voting
C001067,Clarke (NY),NY,D,6.0,41,9.3,Nay,Nay,Yea,Nay,Nay,Nay,Nay,Yea,Yea,No,Nay,Nay,Nay
K000402,Kennedy,NY,D,3.0,77,17.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,Nay,Nay,Nay
T000469,Tonko,NY,D,2.0,100,22.7,Nay,Yea,Yea,Nay,Yea,Nay,Nay,Yea,Yea,No,Nay,Nay,Nay
M001137,Meeks,NY,D,2.0,100,22.7,Nay,Yea,Yea,Nay,Yea,Nay,Nay,Yea,Yea,No,Nay,Nay,Nay
N000002,Nadler,NY,D,2.0,100,22.7,Nay,Yea,Yea,Nay,Nay,Nay,Nay,Yea,Yea,No,Nay,Not Voting,Not Voting
H001038,Higgins (NY),NY,D,2.0,100,22.7,Nay,Yea,NaN,Nay,NaN,Nay,NaN,NaN,NaN,NaN,NaN,NaN,NaN
